<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/main/notebooks/Analyse%20NY%20Compare%20Completely%20Home%20Percentage%20For%20Wealthiest%20And%20Poorest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse NY Compare Completely Home Percentage For Wealthiest And Poorest


**Instructions:**

1. Execute the first code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 1.99 ms (started: 2021-05-03 19:45:53 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 46.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=ab2aa793493df22412aa924eeb01dc44373f745ee738231631202c8f72b3f491
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvi

In [3]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 111 ms (started: 2021-05-03 19:46:56 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 282 ms (started: 2021-05-03 19:46:56 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.01 s (started: 2021-05-03 19:46:56 +00:00)


In [6]:
#sc.stop()

time: 1.43 ms (started: 2021-05-03 19:47:01 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 22.2 s (started: 2021-05-03 19:47:01 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 2.7 ms (started: 2021-05-03 19:47:23 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+
|         cbg|                date|device_count|completely_home_device_count|
+------------+--------------------+------------+----------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|
|360810384001|2020-02-03T00:00:...|          84|                          21|
|360850170103|2020-02-03T00:00:...|         250|                          68|
|360050213021|2020-02-03T00:00:...|          92|                          26|
|360050253004|2020-02-03T00:00:...|          52|                          13|
|360470385003|2020-02-03T00:00:...|          37|                           9|
|360470406002|2020-02-03T00:00:...|          46|                          12|
|360470428003|2020-02-03T00:00:...|          84|                          20|
|360471130002|2020-02-03T00:00:...|          84|                          20|
|360811029001|2020-02-03T00:00:...|         256|                

In [11]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 68.8 ms (started: 2021-05-03 19:47:56 +00:00)


In [12]:
df_soc.show()

+------------+----------+------------+----------------------------+
|         cbg|      date|device_count|completely_home_device_count|
+------------+----------+------------+----------------------------+
|360470064002|2020-02-03|          64|                          18|
|360810384001|2020-02-03|          84|                          21|
|360850170103|2020-02-03|         250|                          68|
|360050213021|2020-02-03|          92|                          26|
|360050253004|2020-02-03|          52|                          13|
|360470385003|2020-02-03|          37|                           9|
|360470406002|2020-02-03|          46|                          12|
|360470428003|2020-02-03|          84|                          20|
|360471130002|2020-02-03|          84|                          20|
|360811029001|2020-02-03|         256|                          95|
|360050177022|2020-02-03|          53|                          19|
|360050248002|2020-02-03|          63|          

In [13]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)


time: 58.9 ms (started: 2021-05-03 19:47:56 +00:00)


In [14]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 24.1 ms (started: 2021-05-03 19:47:56 +00:00)




---


**Demographic data**





---



In [15]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050175002|     1110| 55.22522522522523|14.774774774774773|
|360050141001|     1445| 36.74740484429066| 19.16955017301038|
|360050145001|     2003| 39.47667804323095|  8.53719420868697|
|360050075002|     2731|22.812156719150494|4.5404613694617355|
|360050418001|      936|18.859649122807017| 9.081196581196581|
|360050125001|     2935| 43.24693042291951|12.742759795570699|
|360050127012|     1061| 43.73232799245994|18.378887841658813|
|360050054003|     1031| 57.12900096993211| 28.12803103782735|
|360050027011|     1364|  65.9090909090909|14.589442815249267|
|360050458001|     1168| 37.15753424657534| 2.910958904109589|
|360050224011|     1291|19.907048799380327|27.420604182804027|
|360050389001|     1143| 53.94265232974911|30.708661417322837|
|360050071002|     1661|           37.4375|11.499096929

In [16]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+----------------------------+--------------------------+------------------+
|         cbg|      date|device_count|completely_home_device_count|completely_home_percentage|poverty_percentage|
+------------+----------+------------+----------------------------+--------------------------+------------------+
|360470064002|2020-02-03|          64|                          18|                    28.125| 19.22705314009662|
|360810384001|2020-02-03|          84|                          21|                      25.0|12.291350531107739|
|360850170103|2020-02-03|         250|                          68|        27.200000000000003|13.652802893309223|
|360050213021|2020-02-03|          92|                          26|         28.26086956521739| 43.07328605200946|
|360050253004|2020-02-03|          52|                          13|                      25.0|24.824120603015075|
|360470385003|2020-02-03|          37|                           9|        24.3243243243

In [17]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.completely_home_percentage as top20_perc,
l.completely_home_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 377 ms (started: 2021-05-03 19:47:58 +00:00)


**Load 2019 stats**

In [18]:
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][5]


+-----+------------------+------------------+-----------------+------------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+-----------------+
|  _c0|             total|   completely_home|   part_time_work|    full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|            month|
+-----+------------------+------------------+-----------------+------------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+-----------------+
|count|         2290248.0|         2290248.0|        2290248.0|         2290248.0|                 2290248.0|                2290248.0|                2290248.0|           2290248.0|           2290248.0|           2290248.0|        2290248.0|
| mean| 80.34526610218632|28

In [19]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 36.3 ms (started: 2021-05-03 19:48:00 +00:00)


In [20]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 32.4 ms (started: 2021-05-03 19:48:00 +00:00)



**Visualizations** 



In [21]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 37.1 ms (started: 2021-05-03 19:48:00 +00:00)


In [22]:
def create_chart(df,mon) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','completely_home_value']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = mon),
      y= alt.Y('completely_home_value:Q', title = 'Completely Home %'),
      color='Poverty %:N'
    ).properties(width = 1000).interactive()

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 27.1 ms (started: 2021-05-03 19:48:00 +00:00)


In [23]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 2.69 ms (started: 2021-05-03 19:48:00 +00:00)


In [24]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,20.948572,20.489095,-13.243203,-13.702680
1,2020-04-13,57.527185,47.815293,23.335410,13.623518
2,2020-06-24,39.998245,40.032680,5.806470,5.840905
3,2020-06-08,42.439567,31.688312,8.247792,-2.503463
4,2020-09-12,33.992552,15.840081,-0.199223,-18.351694
...,...,...,...,...,...
361,2020-04-05,54.866796,40.241935,20.675021,6.050161
362,2020-05-01,53.816496,52.280702,19.624721,18.088927
363,2020-10-25,36.063622,28.735632,1.871847,-5.456143
364,2020-12-28,34.252238,34.923077,0.060463,0.731302


time: 3min 32s (started: 2021-05-03 19:48:00 +00:00)


In [25]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 6.28 ms (started: 2021-05-03 19:51:32 +00:00)


In [26]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','completely home %']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('completely home %:Q', title = 'Completely Home %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 151 ms (started: 2021-05-03 19:51:32 +00:00)


In [27]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 4.06 ms (started: 2021-05-03 19:51:33 +00:00)


Filtering based on important dates

**March 8, 2020:**
NYC issues guidelines to avoid densely packed buses, subways, or trains.

**March 12, 2020:**
Broadway shuts down.

**March 14, 2020:**
First two COVID-19 deaths in NYS.

**March 22, 2020:**
NYS on Pause Program begins, all non-essential workers must stay home.

**March 28, 2020:**
Governor Cuomo halts all nonessential construction sites in NYS.











In [28]:
start_date = '2020-3-5'
end_date = '2020-3-31'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1,'March')
chart

alt.LayerChart(...)

time: 104 ms (started: 2021-05-03 19:51:33 +00:00)




**April 10, 2020:**
NYS records more COVID19 cases than any other country 









In [29]:
start_date = '2020-4-2'
end_date = '2020-4-24'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2,'April')
chart
#df2

alt.LayerChart(...)

time: 59.1 ms (started: 2021-05-03 19:51:33 +00:00)



**July 6, 2020:** 
NYC begins Phase 3 of reopening, without indoor dining





In [32]:
start_date = '2020-7-1'
end_date = '2020-7-16'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3,'July')
chart

alt.LayerChart(...)

time: 90.4 ms (started: 2021-05-03 19:58:37 +00:00)


**October 28, 2020:** New York tops a half-million confirmed COVID-19 cases


In [31]:
start_date = '2020-10-20'
end_date = '2020-11-8'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4,'October-November')
chart

alt.LayerChart(...)

time: 63.4 ms (started: 2021-05-03 19:51:33 +00:00)
